In [306]:
import matplotlib as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn import datasets, linear_model, metrics 
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR

In [307]:
#Reading in the datasets

In [308]:
train = pd.read_csv('train.csv')
mi = pd.read_csv('meal_info.csv')
fci = pd.read_csv('fulfilment_center_info.csv')
test = pd.read_csv('test.csv')

In [309]:
train=train.loc[train['num_orders']<2000,:]
fci.describe()

,center_id,city_code,region_code,op_area
count,77.000000,77.000000,77.000000,77.000000
mean,83.142857,600.662338,56.493506,3.985714
std,46.090219,66.720274,18.126473,1.106406
min,10.000000,456.000000,23.000000,0.900000
25%,50.000000,553.000000,34.000000,3.500000
50%,77.000000,596.000000,56.000000,3.900000
75%,110.000000,651.000000,77.000000,4.400000
max,186.000000,713.000000,93.000000,7.000000


In [310]:
#Combining tables to create final table

In [311]:
meal_merge = pd.merge(train,
                 mi[['meal_id','category', 'cuisine']],
                 on='meal_id')

final_df = pd.merge(meal_merge,
                    fci[['center_id','city_code', 'region_code','center_type','op_area']],
                    on='center_id')

In [312]:
final_df.shape

(453148, 15)

In [313]:
final_df = final_df.drop(['center_id','meal_id','id'], axis=1)

In [314]:
final_df['city_code']=final_df['city_code'].astype(str)

In [315]:
final_df['week']=final_df['week']%52

In [316]:
final_df['week']=final_df['week'].astype(str)

In [317]:
wcc_mean=final_df.groupby(['week','city_code','category'],as_index=False).agg({'num_orders':'mean'})

In [318]:
wcc_join=pd.merge(final_df,wcc_mean,on=('week','city_code','category'))

In [319]:
wcc_join.shape

(453148, 13)

In [320]:
final_df=wcc_join

In [321]:
final_df=final_df.drop(['week','category','city_code'],axis=1)

In [322]:
final_df.columns=['checkout_price', 'base_price', 'emailer_for_promotion',
       'homepage_featured', 'num_orders', 'cuisine', 'region_code',
       'center_type', 'op_area', 'num_orders_mean']

In [323]:
#Feature Engineering

In [324]:
final_df['Discount']=final_df['base_price']-final_df['checkout_price']

In [325]:
final_df.loc[final_df['Discount'] < 0, 'Discount'] = 0

In [326]:
final_df = pd.get_dummies(final_df,drop_first=True)

In [327]:
final_df.describe()

,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,region_code,op_area,num_orders_mean,Discount,cuisine_Indian,cuisine_Italian,cuisine_Thai,center_type_TYPE_B,center_type_TYPE_C
count,453148.000000,453148.000000,453148.000000,453148.000000,453148.000000,453148.000000,453148.000000,453148.000000,453148.000000,453148.000000,453148.000000,453148.000000,453148.000000,453148.000000
mean,333.260200,354.772133,0.076880,0.104637,240.709393,56.600691,4.077532,240.709393,21.984518,0.245944,0.267954,0.259308,0.205483,0.218628
std,152.996306,160.992740,0.266401,0.306085,286.852995,17.664059,1.087755,178.505034,47.897670,0.430646,0.442894,0.438256,0.404054,0.413316
min,2.970000,55.350000,0.000000,0.000000,13.000000,23.000000,0.900000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,231.830000,243.500000,0.000000,0.000000,54.000000,34.000000,3.600000,87.739130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,298.760000,311.430000,0.000000,0.000000,135.000000,56.000000,4.000000,196.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,445.230000,463.720000,0.000000,0.000000,312.000000,77.000000,4.500000,363.163194,6.850000,0.000000,1.000000,1.000000,0.000000,0.000000
max,866.270000,866.270000,1.000000,1.000000,1999.000000,93.000000,7.000000,1447.750000,561.600000,1.000000,1.000000,1.000000,1.000000,1.000000


In [328]:
#####Creating test data set##########

In [329]:
test_merge1 = pd.merge(test,
                 mi[['meal_id','category', 'cuisine']],
                 on='meal_id')

final_test = pd.merge(test_merge1,
                    fci[['center_id','city_code', 'region_code','center_type','op_area']],
                    on='center_id')
final_test.shape


(32573, 14)

In [330]:
final_test = final_test.drop(['center_id','meal_id'], axis=1)

In [331]:
final_test['city_code']=final_test['city_code'].astype(str)

In [332]:
final_test['week']=final_test['week']%52

In [333]:
final_test['week']=final_test['week'].astype(str)

In [334]:
final_test.shape

(32573, 12)

In [335]:
final_test=pd.merge(final_test,wcc_mean,on=('week','city_code','category'),how='left')

In [336]:
final_test.shape

(32573, 13)

In [337]:
null_data = final_test[final_test.isnull().any(axis=1)]

In [338]:
null_data.head()

,id,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,category,cuisine,city_code,region_code,center_type,op_area,num_orders
4371,1091512,45,427.83,425.83,0,0,Biryani,Indian,649,34,TYPE_A,3.4,NaN
4394,1346923,45,446.23,447.23,0,0,Biryani,Indian,649,34,TYPE_A,3.4,NaN
5057,1260468,47,280.33,321.13,0,0,Other Snacks,Thai,541,77,TYPE_C,2.8,NaN
5061,1287575,51,319.13,319.13,0,0,Other Snacks,Thai,541,77,TYPE_C,2.8,NaN
5143,1052283,45,425.83,484.03,0,0,Biryani,Indian,541,77,TYPE_C,2.8,NaN


In [339]:
final_test['num_orders'].fillna((final_test['num_orders'].mean()), inplace=True)

In [340]:
final_test=final_test.drop(['week','category','city_code'],axis=1)

In [341]:
final_test.shape

(32573, 10)

In [342]:
#Feature Engineering

In [343]:
final_test['Discount']=final_test['base_price']-final_test['checkout_price']

In [344]:
final_test.loc[(final_test['Discount'] <= 0),'Discount'] = 0

In [345]:
final_test = pd.get_dummies(final_test,drop_first=True)

In [346]:
final_test.describe(include='all')

,id,checkout_price,base_price,emailer_for_promotion,homepage_featured,region_code,op_area,num_orders,Discount,cuisine_Indian,cuisine_Italian,cuisine_Thai,center_type_TYPE_B,center_type_TYPE_C
count,3.257300e+04,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000
mean,1.248476e+06,341.854440,356.493615,0.066435,0.081356,56.712154,4.087726,241.801129,15.100768,0.239677,0.271268,0.256102,0.210235,0.215178
std,1.441580e+05,153.893886,155.150101,0.249045,0.273385,17.641174,1.098835,175.763775,39.160919,0.426893,0.444621,0.436485,0.407482,0.410952
min,1.000085e+06,67.900000,89.240000,0.000000,0.000000,23.000000,0.900000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.123969e+06,214.430000,243.500000,0.000000,0.000000,34.000000,3.600000,92.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.247296e+06,320.130000,321.130000,0.000000,0.000000,56.000000,4.000000,200.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.372971e+06,446.230000,455.930000,0.000000,0.000000,77.000000,4.500000,362.760870,2.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,1.499996e+06,1113.620000,1112.620000,1.000000,1.000000,93.000000,7.000000,1037.000000,820.590000,1.000000,1.000000,1.000000,1.000000,1.000000


In [347]:
#########Seperating by regions based on visualizations##########
#########Creating models for each region#########

In [348]:
########## Region 56 #############

In [349]:
r56 = final_df[final_df.region_code == 56]
r56.shape

(189013, 14)

In [350]:
#Splitting data into training and testing sets

In [351]:
X = r56.copy().drop(["num_orders",'region_code'],axis=1)
y = r56["num_orders"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [352]:
r56_rf = RandomForestRegressor(n_estimators=100, random_state=0)  
r56_rf.fit(X_train, y_train)  
y_pred = r56_rf.predict(X_test)

In [353]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Root Mean Squared Error: 160.05217261354267


In [354]:
r56.head()

,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,region_code,op_area,num_orders_mean,Discount,cuisine_Indian,cuisine_Italian,cuisine_Thai,center_type_TYPE_B,center_type_TYPE_C
0,136.83,152.29,0,0,177,56,2.0,523.898551,15.46,0,0,1,0,1
1,122.28,145.53,1,1,500,56,2.0,523.898551,23.25,0,0,1,0,1
2,150.41,148.41,0,0,229,56,2.0,523.898551,0.00,0,0,1,0,1
3,136.83,135.83,0,0,270,56,2.0,523.898551,0.00,0,0,1,0,1
4,120.28,145.56,1,1,1161,56,2.0,523.898551,25.28,0,0,1,0,1


In [355]:
#Predicting on Test

In [356]:
r56_test = final_test[final_test.region_code == 56]

r56_test_score=r56_test.drop(['id','region_code'],axis=1)
r56_test_score.head()


,checkout_price,base_price,emailer_for_promotion,homepage_featured,op_area,num_orders,Discount,cuisine_Indian,cuisine_Italian,cuisine_Thai,center_type_TYPE_B,center_type_TYPE_C
0,158.11,159.11,0,0,2.0,373.106383,1.0,0,0,1,0,1
1,159.11,159.11,0,0,2.0,329.531915,0.0,0,0,1,0,1
2,157.14,158.14,0,0,2.0,339.875000,1.0,0,0,1,0,1
3,159.14,157.14,0,0,2.0,338.166667,0.0,0,0,1,0,1
4,160.11,159.11,0,0,2.0,326.979167,0.0,0,0,1,0,1


In [357]:
num_orders_r56 = r56_rf.predict(r56_test_score)

In [358]:
num_orders_r56_results = np.ceil(num_orders_r56)

In [359]:
num_orders_r56_results

array([182., 124.,  99., ..., 124., 136.,  84.])

In [360]:
final_test.head()

,id,checkout_price,base_price,emailer_for_promotion,homepage_featured,region_code,op_area,num_orders,Discount,cuisine_Indian,cuisine_Italian,cuisine_Thai,center_type_TYPE_B,center_type_TYPE_C
0,1028232,158.11,159.11,0,0,56,2.0,373.106383,1.0,0,0,1,0,1
1,1262649,159.11,159.11,0,0,56,2.0,329.531915,0.0,0,0,1,0,1
2,1453211,157.14,158.14,0,0,56,2.0,339.875000,1.0,0,0,1,0,1
3,1262599,159.14,157.14,0,0,56,2.0,338.166667,0.0,0,0,1,0,1
4,1495848,160.11,159.11,0,0,56,2.0,326.979167,0.0,0,0,1,0,1


In [361]:
sol_df_r56 = pd.DataFrame({'id':r56_test.id, 'num_orders': num_orders_r56_results})

In [362]:
sol_df_r56.shape

(13683, 2)

In [363]:
############Region 85###################

In [364]:
r85 = final_df[final_df.region_code == 85]

In [365]:
X = r85.copy().drop(["num_orders","region_code"],axis=1)
y = r85["num_orders"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [366]:
r85_rf = RandomForestRegressor(n_estimators=100, random_state=0)  
r85_rf.fit(X_train, y_train)  
y_pred = r85_rf.predict(X_test)

In [367]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Root Mean Squared Error: 155.78387746334838


In [368]:
#Predicting on Test

In [369]:
r85_test = final_test[final_test.region_code == 85]

r85_test_score=r85_test.drop(['id','region_code'],axis=1)
r85_test.head()


,id,checkout_price,base_price,emailer_for_promotion,homepage_featured,region_code,op_area,num_orders,Discount,cuisine_Indian,cuisine_Italian,cuisine_Thai,center_type_TYPE_B,center_type_TYPE_C
362,1460444,158.11,160.11,0,1,85,3.6,369.288889,2.0,0,0,1,1,0
363,1476535,157.14,157.14,0,0,85,3.6,362.844444,0.0,0,0,1,1,0
364,1297121,160.11,160.11,0,0,85,3.6,356.955556,0.0,0,0,1,1,0
365,1141103,158.14,158.14,0,0,85,3.6,339.767442,0.0,0,0,1,1,0
366,1274933,159.11,158.11,0,0,85,3.6,371.266667,0.0,0,0,1,1,0


In [370]:
num_orders_r85 = r85_rf.predict(r85_test_score)

In [371]:
num_orders_r85_results = np.ceil(num_orders_r85)

In [372]:
num_orders_r85_results

array([1105.,  942.,  690., ...,   35.,   19.,   33.])

In [373]:
sol_df_r85 = pd.DataFrame({'id':r85_test.id, 'num_orders': num_orders_r85_results})

In [374]:
sol_df_r85.shape

(2238, 2)

In [375]:
##################Region 34 and 77#################################

In [376]:
r3477 = final_df[(final_df.region_code == 34)  | (final_df.region_code == 77)]

In [377]:
X = r3477.copy().drop("num_orders",axis=1)
y = r3477["num_orders"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [378]:
r3477_rf = RandomForestRegressor(n_estimators=100, random_state=0)  
r3477_rf.fit(X_train, y_train)  
y_pred = r3477_rf.predict(X_test)

In [379]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Root Mean Squared Error: 113.49847890864984


In [380]:
#Predicting on Test

In [381]:
r3477_test = final_test[(final_test.region_code == 34)  | (final_test.region_code == 77)]

r3477_test_score=r3477_test.drop(['id'],axis=1)
r3477_test.shape


(14953, 14)

In [382]:
num_orders_r3477 = r3477_rf.predict(r3477_test_score)

In [383]:
num_orders_r3477_results = np.ceil(num_orders_r3477)

In [384]:
num_orders_r3477_results

array([607., 454., 586., ...,  52.,  37.,  86.])

In [385]:
sol_df_r3477 = pd.DataFrame({'id':r3477_test.id, 'num_orders': num_orders_r3477_results})

In [386]:
sol_df_r3477.shape

(14953, 2)

In [387]:
##################Region 35, 71, 93 and 23#################################

In [388]:
r_other = final_df[(final_df.region_code == 35)  | (final_df.region_code == 71) | (final_df.region_code == 93) | (final_df.region_code == 23)]

In [389]:
X = r_other.copy().drop("num_orders",axis=1)
y = r_other["num_orders"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [390]:
r_other_rf = RandomForestRegressor(n_estimators=100, random_state=0)  
r_other_rf.fit(X_train, y_train)  
y_pred = r_other_rf.predict(X_test)

In [391]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Root Mean Squared Error: 134.9406188310536


In [392]:
#Predicting on Test

In [393]:
r_other_test = final_test[(final_test.region_code == 35)  | (final_test.region_code == 71) | (final_test.region_code == 93) | (final_test.region_code == 23)]

r_other_test_score=r_other_test.drop(['id'],axis=1)
r_other_test.shape


(1699, 14)

In [394]:
num_orders_r_other = r_other_rf.predict(r_other_test_score)

In [395]:
num_orders_r_other_results = np.ceil(num_orders_r_other)

In [396]:
num_orders_r_other_results

array([190., 116., 126., ...,  20.,  15.,  15.])

In [397]:
sol_df_rother = pd.DataFrame({'id':r_other_test.id, 'num_orders': num_orders_r_other_results})

In [398]:
sol_df_rother.shape

(1699, 2)

In [399]:
final_test[(final_test.region_code == 35)  | (final_test.region_code == 71) | (final_test.region_code == 93) | (final_test.region_code == 23)]

,id,checkout_price,base_price,emailer_for_promotion,homepage_featured,region_code,op_area,num_orders,Discount,cuisine_Indian,cuisine_Italian,cuisine_Thai,center_type_TYPE_B,center_type_TYPE_C
7256,1312130,121.31,121.31,0,0,35,2.8,170.208333,0.0,0,0,1,0,0
7257,1284072,120.28,120.28,0,0,35,2.8,167.583333,0.0,0,0,1,0,0
7258,1390606,121.28,120.28,0,0,35,2.8,130.541667,0.0,0,0,1,0,0
7259,1177870,119.34,119.34,0,0,35,2.8,144.500000,0.0,0,0,1,0,0
7260,1217412,118.34,119.34,0,0,35,2.8,133.739130,1.0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30795,1452741,446.23,446.23,0,0,23,3.4,21.250000,0.0,1,0,0,0,0
30796,1468212,446.23,447.23,0,0,23,3.4,15.000000,1.0,1,0,0,0,0
30797,1230784,445.23,447.23,0,0,23,3.4,20.500000,2.0,1,0,0,0,0
30798,1190650,445.23,445.23,0,0,23,3.4,14.666667,0.0,1,0,0,0,0


In [400]:
final_test.groupby('region_code').count()

,id,checkout_price,base_price,emailer_for_promotion,homepage_featured,op_area,num_orders,Discount,cuisine_Indian,cuisine_Italian,cuisine_Thai,center_type_TYPE_B,center_type_TYPE_C
region_code,,,,,,,,,,,,,
23,449,449,449,449,449,449,449,449,449,449,449,449,449
34,8265,8265,8265,8265,8265,8265,8265,8265,8265,8265,8265,8265,8265
35,370,370,370,370,370,370,370,370,370,370,370,370,370
56,13683,13683,13683,13683,13683,13683,13683,13683,13683,13683,13683,13683,13683
71,468,468,468,468,468,468,468,468,468,468,468,468,468
77,6688,6688,6688,6688,6688,6688,6688,6688,6688,6688,6688,6688,6688
85,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238
93,412,412,412,412,412,412,412,412,412,412,412,412,412


In [401]:
sol_df_r3477.shape

(14953, 2)

In [402]:
final_sol = pd.concat([sol_df_r56,sol_df_r85,sol_df_r3477,sol_df_rother])

In [403]:
final_sol.to_csv('results_final.csv',index=False)

In [404]:
final_sol.shape

(32573, 2)